In [4]:
import pandas as pd # data handling library
from pylab import rcParams
from mpl_toolkits.basemap import Basemap # world map
import matplotlib.pyplot as plt # python plotting library
import numpy as np # numerical python

rcParams['figure.figsize'] = (8,6)

In [5]:
# load cell coordinates
cells = pd.read_csv('data/cell5m_allockey_xy.csv', sep=',', header=0)

In [5]:
# load spatially resolved harvested area
spam = pd.read_csv('data/spam2005v2r0_harvested-area_wheat_total.csv', sep=',', header=0)
spam_ethiopia = spam[spam.name_cntr=='Ethiopia'] # reduce to Ethiopia
spam_ethiopia = spam_ethiopia.loc[:,['cell5m','whea']] # reduce #columns dimensions

In [6]:
# load historical national harvest data
history = pd.read_csv('data/FAOSTAT_data_8-1-2017.csv', sep=',', header=0)
history_ethiopia = history[(history.Area=='Ethiopia') | (history.Area=='Ethiopia PDR')] # reduce to Ethiopia

In [7]:
# Get x-y coordinates from cell number (takes a while)
X = np.zeros(len(spam_ethiopia)) # vector for longitudes
Y = np.zeros(len(spam_ethiopia)) # vector for latitudes
C = np.zeros(len(spam_ethiopia)) # vector for harvest data [ha]

k=0 # iteration variable
N = len(spam_ethiopia)

print "this might take a while"
for index, row in spam_ethiopia.iterrows(): # iterate over rows in SPAM data
    coordinates = cells[cells.hc_seq5m==row['cell5m']] # find coordinates for each cell
    X[k] = coordinates['x'].item()
    Y[k] = coordinates['y'].item()
    C[k] = row['whea'].item()
    k+=1
    if k%1000==0: # show progress
        print k,"/",N
print "done"

this might take a while
1000 / 4651
2000 / 4651
3000 / 4651
4000 / 4651
done


In [14]:
# harvest in 2014 according to FAO data
harv_2014_fao = history_ethiopia[history_ethiopia.Year == 2014].Value.item()
harv_2005_fao = history_ethiopia[history_ethiopia.Year == 2005].Value.item()

# total harvest in 2005 according to SPAM data
harv_2005_spam = spam_ethiopia['whea'].sum()

scaling_factor = harv_2014_fao/harv_2005_spam # ratio of harvest in 2014 and 2005

D = C*scaling_factor # multiply every harvest value from 2005 by the scaling factor above
                     # so that the total harvest fits the one of 2014

harv_2014_downscaled = D.sum() # total harvest of downscaled data

# comparison of the dofferent data sources:

print "FAO:", "2014:", harv_2014_fao, "\t2005:", harv_2005_fao
print "SPAM:", "2005:", harv_2005_spam
print "downscaled 2014:", harv_2014_downscaled

# check whether downscaled data is consistent with FAO (it is by construction)
if harv_2014_fao==harv_2014_downscaled: 
    print "\nDownscaling consitent with FAO data."
else:
    print "\nDownscaling inconsistent!"        

FAO: 2014: 1663845.0 	2005: 1398215.0
SPAM: 2005: 1318884.4
downscaled 2014: 1663845.0

Downscaling consitent with FAO data.


In [29]:
# Plotting Ex. 1

#draw the border of ethiopia using latitude and longitude)
plt.figure()
my_map = Basemap(projection='merc', lat_0=57, lon_0=-135,
    resolution = 'l', area_thresh = 1000.0,
    llcrnrlon=30.25, llcrnrlat=0,
    urcrnrlon=50.25, urcrnrlat=15)
 
my_map.drawcoastlines()
my_map.drawcountries()
my_map.drawmapboundary()

# Use the coordinates (X,Y) and wheat production (D) to draw a color coded map.
my_map.scatter(X, Y, latlon = True, c=D, cmap='YlOrRd', marker='s', s=8)
plt.colorbar(label='wheat production in ha')

In [30]:
# Plotting Ex. 2

plt.figure()
history_sorted = history_ethiopia.sort_values(by='Year') # sort data by year

plt.plot(history_sorted.Year,history_sorted.Value, "o-") # plot historical data

plt.title("Annual wheat production of Ethiopia according to FAO")
plt.xlabel("year")
plt.ylabel("wheat production in ha")
plt.axvline(x=1974, ls="--")
plt.axvline(x=1991, ls="--")
plt.text(1960,1200000,"Haile Selassie I era")
plt.text(1980,900000,"Derg era")
plt.text(1995,600000,"Federal Democratic Republic")

In [31]:
plt.show()